In [431]:
import sys
import os

try:
    from diffform import *
except ImportError as e:
    sys.path.append(os.getcwd()+'/..')
    from core import *


from sympy import symbols, sin, Function, I, Array, permutedims, Eq, solve, cos, simplify, sqrt

set_max_degree(4)

coords = t,r,theta,phi = symbols(r"t r \theta \phi")
basis = dt,dr,dtheta,dphi = d(t), d(r), d(theta), d(phi)

# f = sqrt(1-1/r)
f = Function("f")(r)

# g = 1/f
g = Function("g")(r)

e0 = f*dt
e1 = g*dr
e2 = r*dtheta
e3 = r*sin(theta)*dphi

S1 = I*e0*e1 - e2*e3
S2 = I*e0*e2 - e3*e1
S3 = I*e0*e3 - e1*e2

AS1 = I*e0*e1 + e2*e3
AS2 = I*e0*e2 + e3*e1
AS3 = I*e0*e3 + e1*e2

A_symbols = permutedims(Array([symbols(f"A^{{1:4}}_{{{str(c)}}}") for c in coords]),(1,0))

A1 = sum([A_symbols[0,i]*basis[i] for i in range(4)])
A2 = sum([A_symbols[1,i]*basis[i] for i in range(4)])
A3 = sum([A_symbols[2,i]*basis[i] for i in range(4)])

# A1 = I*f.diff(r)/g*dt + cos(theta)*dphi 
# A2 = -1/g * sin(theta) * dphi
# A3 = 1/g * dtheta

dAS1 = S1.d + A2*S3 - A3*S2
dAS2 = S2.d + A3*S1 - A1*S3
dAS3 = S3.d + A1*S2 - A2*S1

dA_equations = dAS1.factors + dAS2.factors + dAS3.factors

A_sol = solve(dA_equations)[-1]

A1 = A1.subs(A_sol)
A2 = A2.subs(A_sol)
A3 = A3.subs(A_sol)

print("")
print("Self-Dual Connection")
display(A1)
display(A2)
display(A3)

F1 = A1.d + A2*A3
F2 = A2.d + A3*A1
F3 = A3.d + A1*A2

FS  = [F1,F2,F3]
S  = [S1,S2,S3]
AS = [AS1,AS2,AS3]

Ricci = Matrix([[0 if len((FS[i]*AS[j]).factors) == 0 else simplify((FS[i]*AS[j]).factors[0]/(AS[0]*AS[0]).factors[0]) for j in range(3)] for i in range(3)])
Weyl  = Matrix([[0 if len((FS[i]*S[j]).factors) == 0 else simplify((FS[i]*S[j]).factors[0]/(S[0]*S[0]).factors[0]) for j in range(3)] for i in range(3)])

print("")
print("Self-Dual Weyl")
display(Weyl)
print("Self-Dual Ricci")
display(Ricci)


Self-Dual Connection



Self-Dual Weyl


Matrix([
[-(r**2*(g(r)*Derivative(f(r), (r, 2)) - Derivative(f(r), r)*Derivative(g(r), r)) - (g(r)**2 - 1)*f(r)*g(r))/(2*r**2*f(r)*g(r)**3),                                                                        0,                                                                        0],
[                                                                                                                                0, (f(r)*Derivative(g(r), r) - g(r)*Derivative(f(r), r))/(2*r*f(r)*g(r)**3),                                                                        0],
[                                                                                                                                0,                                                                        0, (f(r)*Derivative(g(r), r) - g(r)*Derivative(f(r), r))/(2*r*f(r)*g(r)**3)]])

Self-Dual Ricci


Matrix([
[-(r**2*(g(r)*Derivative(f(r), (r, 2)) - Derivative(f(r), r)*Derivative(g(r), r)) + (g(r)**2 - 1)*f(r)*g(r))/(2*r**2*f(r)*g(r)**3),                                                                         0,                                                                         0],
[                                                                                                                                0, -(f(r)*Derivative(g(r), r) + g(r)*Derivative(f(r), r))/(2*r*f(r)*g(r)**3),                                                                         0],
[                                                                                                                                0,                                                                         0, -(f(r)*Derivative(g(r), r) + g(r)*Derivative(f(r), r))/(2*r*f(r)*g(r)**3)]])

In [428]:
import sys
import os

try:
    from diffform import *
except ImportError as e:
    sys.path.append(os.getcwd()+'/..')
    from core import *

from sympy import symbols, Function, I, Array, solve, simplify, sqrt

set_max_degree(4)

coords = a,x,y,z = symbols(r"\alpha x y z")
basis = alpha,dx,dy,dz = DifferentialForm(a,1),d(x),d(y),d(z)

w = Function("w")(x,y,z)
u = Function("u")(x,y,z)

dalpha_sub = {d(alpha): w.diff(x)*dy*dz + w.diff(y)*dz*dx + (exp(u/2)*w).diff(z)*dx*dy}

# n = symbols("n",integer=True)

e0 = 1/(z*sqrt(w))*alpha
e1 = sqrt(w)/z*dz
e2 = 1/z*exp(u/2)*sqrt(w)*dx
e3 = 1/z*exp(u/2)*sqrt(w)*dy

S1 = e0*e1-e2*e3
S2 = e0*e2-e3*e1
S3 = e0*e3-e1*e2

AS1 = e0*e1+e2*e3
AS2 = e0*e2+e3*e1
AS3 = e0*e3+e1*e2

A_symbols = symbols(r"A^{1:4}_{0:4}")

A1 = sum([A_symbols[(1-1)*4+I]*basis[I] for I in range(4)])
A2 = sum([A_symbols[(2-1)*4+I]*basis[I] for I in range(4)])
A3 = sum([A_symbols[(3-1)*4+I]*basis[I] for I in range(4)])

A_eqns = simplify(S1.d.subs(dalpha_sub) + A2*S3-A3*S2).factors + simplify(S2.d.subs(dalpha_sub) + A3*S1-A1*S3).factors + simplify(S3.d.subs(dalpha_sub) + A1*S2-A2*S1).factors

A_sol = solve(A_eqns,A_symbols)

A1 = simplify(A1.subs(A_sol))
A2 = simplify(A2.subs(A_sol))
A3 = simplify(A3.subs(A_sol))

display(A1)
display(A2)
display(A3)


In [432]:
# Double Complex Structure Example

import sys
import os

try:
    from diffform import *
except ImportError as e:
    sys.path.append(os.getcwd()+'/..')
    from core import *

from sympy import symbols, exp, simplify, solve, sqrt, Matrix

set_max_degree(4)

x,y = symbols(r"x y")
betas = b0,b1,b2,b3 = [DifferentialForm(symbols(rf"\beta_{{{i}}}"),1) for i in range(4)]
tbetas = tb0,tb1,tb2,tb3 = [DifferentialForm(symbols(rf"\hat{{\beta}}_{{{i}}}"),1) for i in range(4)]

b1.exact = True
b2.exact = True

dbeta_subs = {d(x): b1, d(y): b2, d(b0): (1/(x-y))*(b0-b3)*b2, d(b3): (1/(x-y))*(b0-b3)*b1}

F,A = [Function(c)(x) for c in "FA"]
G,B = [Function(c)(y) for c in "GB"]

H = Function("H")(x,y)

e0 = 1/H*F/A/sqrt(x-y) * b0
e1 = 1/H/(F*A)*sqrt(x-y) * b1
e2 = 1/H/(G*B)*sqrt(x-y) * b2
e3 = 1/H*G/B/sqrt(x-y) * b3

S1 = e0*e1 - e2*e3
S2 = e0*e2 - e3*e1
S3 = e0*e3 - e1*e2

S = [S1,S2,S3]

AS1 = e0*e1 + e2*e3
AS2 = e0*e2 + e3*e1
AS3 = e0*e3 + e1*e2

AS = [AS1,AS2,AS3]

A_symbols = symbols(r"A^{1:4}_{0:4}")

A1 = sum([A_symbols[(1-1)*4+I]*betas[I] for I in range(4)])
A2 = sum([A_symbols[(2-1)*4+I]*betas[I] for I in range(4)])
A3 = sum([A_symbols[(3-1)*4+I]*betas[I] for I in range(4)])

A_eqns = simplify(S1.d.subs(dbeta_subs) + A2*S3-A3*S2).factors + simplify(S2.d.subs(dbeta_subs) + A3*S1-A1*S3).factors + simplify(S3.d.subs(dbeta_subs) + A1*S2-A2*S1).factors

A_sol = solve(A_eqns,A_symbols)

A1 = simplify(A1.subs(A_sol))
A2 = simplify(A2.subs(A_sol))
A3 = simplify(A3.subs(A_sol))

apply_J = { b0: ((x-y)/F**2)*tb1, b1: -(F**2/(x-y))*tb0, b2: -(G**2/(x-y))*tb3, b3: ((x-y)/G**2)*tb2, tb0:b0,tb1:b1,tb2:b2,tb3:b3 }


Hx_cond = {H.diff(x): B**2*H/(2*A**2*(x-y))}


F1 = simplify((A1.d + A2*A3).subs(dbeta_subs))
F2 = simplify((A2.d + A3*A1).subs(dbeta_subs))
F3 = simplify((A3.d + A1*A2).subs(dbeta_subs))

FS = [F1,F2,F3]

Ricci = Matrix([[0 if len((FS[i]*AS[j]).factors) == 0 else simplify((FS[i]*AS[j]).factors[0]/(AS[0]*AS[0]).factors[0]) for j in range(3)] for i in range(3)])
Weyl  = Matrix([[0 if len((FS[i]*S[j]).factors) == 0 else simplify((FS[i]*S[j]).factors[0]/(S[0]*S[0]).factors[0]) for j in range(3)] for i in range(3)])


In [372]:
from sympy import expand, ln

R12 = Ricci[0,1]*2*(x**2-2*x*y+y**2)*A*B/(F*G*H)
# display_no_arg(R12)

R21 = Ricci[1,0]*2*A*B*(x**2-2*x*y+y**2)/(F*G*H)


R1p = simplify((R12+R21)/(2*A**2*B**2))
display_no_arg(R1p)

R1m = simplify(R12-R21)/2

display_no_arg(R1m)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [344]:
t = x+y
r = x-y

c0,c1,c2,c3,c4,c5 = symbols(r"c0:6")
r0,r1,r2 = symbols(r"r0:3", real=True,positive=True)

# Plebanski Demianski Solution
c0 = 0
c1 = 0
c3 = 0
c5 = 0
r2 = 0
r0 = 0

# # Other Solution?
# c0 = 0
# c3 = 0
# c2 = 0
# c1 = -4*c3*r1/r2
# r0 = (c5/2)*r1-(c5/2)**2*r2

H0 = simplify(c0*r**2*ln(r) + (c0*t**2+c3*r**2) + c1*t + c2 + c4/2*sqrt((t+c5)**2-r**2))
A0 = sqrt(sqrt(r0 + r1*x + r2*x**2))
B0 = sqrt(sqrt(r0 + r1*y + r2*y**2))

H_cond = simplify(expand(simplify(R1m.subs(A,A0).subs(B,B0).subs(H,H0))))*sqrt(c5**2+2*c5*x+2*c5*y + 4*x*y)*4

In [440]:
W11 = (simplify(Weyl[0,0]))
R11 = (simplify(Ricci[0,0]*8))

display_no_arg(simplify(W11+R11))

<IPython.core.display.Math object>

In [25]:
# Pure Connection Equations
import sys
import os

try:
    from diffform import *
except ImportError as err:
    sys.path.append(os.getcwd()+'/..')
    from core import *

from sympy import symbols, Function, simplify, expand, factor, I, solve, exp, Matrix

set_max_degree(4)

x,y = symbols(r"x y",real=True,positive=True)
betas  = b0,b1,b2,b3     = [DifferentialForm(symbols(rf"\beta_{{{I}}}"),1) for I in range(4)]
tbetas = tb0,tb1,tb2,tb3 = [DifferentialForm(symbols(rf"\hat{{\beta}}_{{{I}}}"),1) for I in range(4)]

b1.exact = True
b2.exact = True

F = Function("F")(x)
G = Function("G")(y)

dbeta_sub = {d(x): b1, d(y): b2, d(b0): (1/(x-y))*(b0-b3)*b2, d(b3): (1/(x-y))*(b0-b3)*b1}

Jm_subs = {b0: (x-y)/F * tb1, b1: -F/(x-y)*tb0, b2: -G/(x-y) * tb3, b3: (x-y)/G * tb2, tb0: b0, tb1: b1, tb2: b2, tb3: b3}

a,b,c,e = [Function(c)(x,y) for c in "abce"]

u = Function("u")(x,y)

h = Function("h")(x,y) # h = Log[f]

c0,c1,c2,c3 = constants(r"c_{0} c_{1} c2 c3")

a = F*u.diff(x)/(x-y)
b = G*u.diff(y)/(x-y)
c = (c0+c1*x)/(F)*exp(u)
e = -(c0+c1*y)/(G)*exp(u)

A1 = simplify(a*b0 + b*b3)
A3 = simplify(c*b1 + e*b2)
A2 = simplify(A3.subs(Jm_subs))

F1 = simplify((A1.d + A2*A3).subs(dbeta_sub))
F2 = simplify((A2.d + A3*A1).subs(dbeta_sub))
F3 = simplify((A3.d + A1*A2).subs(dbeta_sub))
F2.collect_forms()

F1 = F1.subs(u.diff(x,y),(c0+c1*x)*(c0+c1*y)/(F*G)*exp(2*u))

# display(simplify(F1.subs(u.diff(x,y),(c0+c1*x)*(c0+c1*y)/(F*G)*exp(2*u))))
# display(F2)
# display(F3)

FS = [F1,F2,F3]

FF = Matrix([[0 if (FS[i]*FS[j]).factors == [] else simplify((FS[i]*FS[j]).factors[0]) for j in range(3)] for i in range(3)])

pde2 = simplify((2*d(h)*F2 + 3*A3*F1).subs(dbeta_sub))

display(A1)
display(A2)
display(A3)

In [47]:
# U(1) Kerr Background Perturbation
import sys
import os

try:
    from diffform import *
except ImportError as err:
    sys.path.append(os.getcwd()+'/..')
    from core import *

from sympy import symbols, Function, simplify, expand, factor, I, solve, exp, Matrix, sqrt

set_max_degree(4)

t,r,q,vphi = symbols(r"\tau r q \varphi",real=True,positive=True)
betas  = b0,b1,b2,b3     = [DifferentialForm(symbols(rf"\beta_{{{I}}}"),1) for I in range(4)]
tbetas = tb0,tb1,tb2,tb3 = [DifferentialForm(symbols(rf"\hat{{\beta}}_{{{I}}}"),1) for I in range(4)]

b1.exact = True
b2.exact = True

F = Function("F",real=True,positive=True)(r)
G = Function("G",real=True,positive=True)(q)

dbeta_sub = {d(b0): 2*q/(r**2-q**2)*(b0-b3)*b2, d(b3): 2*r/(r**2-q**2)*(b0-b3)*b1, d(r): b1, d(q): b2, d(t): (1/(r**2-q**2))*(r**2*b0-q**2*b3), d(vphi): (1/(r**2-q**2))*(b0-b3)}

hodge_2forms_sub = {b0*b1: tb2*tb3, b0*b2: -F/G*tb0*tb2, b0*b3: (r**2-q**2)**2/(F*G) *tb1*tb2, b1*b2: F*G/(r**2-q**2)**2*tb0*tb3, b1*b3: -F/G*tb0*tb2, b2*b3: tb0*tb1, \
                    tb0*tb1: b0*b1, tb0*tb2: b0*b2, tb0*tb3: b0*b3, tb1*tb2: b1*b2, tb1*tb3: b1*b3, tb2*tb3: b2*b3}

e0 = sqrt(F/(r**2-q**2))*b0
e1 = sqrt((r**2-q**2)/F)*b1
e2 = sqrt((r**2-q**2)/G)*b2
e3 = sqrt(G/(r**2-q**2))*b3

es = [e0,e1,e2,e3]

S1 = simplify(simplify(e0*e1 - e2*e3).subs(sqrt(r**2-q**2),1/sqrt(1/(r**2-q**2))))
S2 = simplify(simplify(e0*e2-e3*e1).subs(sqrt(r**2-q**2),1/sqrt(1/(r**2-q**2))))
S3 = simplify(e0*e3-e1*e2)

Sp = S2+I*S3
Sm = S2-I*S3

f0,f1,f2 = [Function(rf"\phi_{{{i}}}")(r,q) for i in range(3)]

w,k = constants(r"\omega \kappa")


pde1 = simplify(simplify(d(f0*exp(I*(w*t + k*vphi))*Sm+f1*exp(I*(w*t + k*vphi))*S1+f2*exp(I*(w*t + k*vphi))*Sp).subs(dbeta_sub)))

In [48]:
pde1